In [1]:
import pandas as pd
import datetime as dt
import time
import requests


# Reviewer: skip down to 'For Loop. Take III'
Content above that is exploratory testing that I should have done in a separate notebook.  I want to leave it in place for future reference, but I can't rerun the notebook without repeating scraping.  Commenting out the actual 'requests' just means all subsequent dfs and figures don't load...Sorry I did not feng shui this properly in advance!

In [2]:
pd.set_option('display.max_rows', None) #also sorry about this...

In [7]:
url = 'https://api.pushshift.io/reddit/search/submission'
ra = '?subreddit=relatinoship_advice'
pr = '?subreddit=pettyrevenge' #no underscore!!

### Step 1: explore methods, pettyrevenge
#### Takeaways:
* Can only get 100 posts per prequest (lit says 500, it's wrong)
* 77% of posts from test pull were selftext:'removed' , can filter during request
* petty_revenge is the WRONG subreddit, too small.  pettyrevenge no underscore is target.
* relationship_advice is way more active, make sure to sample from similar date ranges (so not all 7,000 r_a posts are from 2021 while pr crosses years - diction evolves).

In [50]:
params = {
    'subreddit' : 'pettyrevenge',
    'size' : 100,
    'selftext:not' : '[removed]' #thanks Amanda!  filters out removed posts (77/100 in test run)
    #'sort' : 'desc'   #default is sort:desc, specify 'asc' to start with oldest
}

In [52]:
# res = requests.get(url, params)

In [53]:
res.status_code

200

In [74]:
json_data=res.json()

In [109]:
res.json()['data'][]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'Celara001',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_6mg7lhlf',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1619163671,
 'domain': 'self.pettyrevenge',
 'full_link': 'https://www.reddit.com/r/pettyrevenge/comments/mwq22g/ignore_me_at_my_old_job_prepare_to_be_ignored/',
 'gildings': {},
 'id': 'mwq22g',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 61,
 'num_crossposts': 0,
 'over_18': False,
 'pare

In [55]:
posts = json_data['data'] #list of posts in section already titled 'data'

In [56]:
len(posts)

100

In [15]:
type(posts)

list

In [57]:
#posts[14] #format = list of dictionaries = easy pd.df

[{'all_awardings': [],
  'allow_live_comments': False,
  'author': 'TexasNelwyn',
  'author_flair_css_class': None,
  'author_flair_richtext': [],
  'author_flair_text': None,
  'author_flair_type': 'text',
  'author_fullname': 't2_3q6nnvt6',
  'author_patreon_flair': False,
  'author_premium': False,
  'awarders': [],
  'can_mod_post': False,
  'contest_mode': False,
  'created_utc': 1619204046,
  'domain': 'self.pettyrevenge',
  'full_link': 'https://www.reddit.com/r/pettyrevenge/comments/mx2cv9/just_desserts_boss_lady_eat_it_anytime/',
  'gildings': {},
  'id': 'mx2cv9',
  'is_crosspostable': True,
  'is_meta': False,
  'is_original_content': False,
  'is_reddit_media_domain': False,
  'is_robot_indexable': True,
  'is_self': True,
  'is_video': False,
  'link_flair_background_color': '',
  'link_flair_richtext': [],
  'link_flair_text_color': 'dark',
  'link_flair_type': 'text',
  'locked': False,
  'media_only': False,
  'no_follow': False,
  'num_comments': 0,
  'num_crossposts':

In [119]:
#filter out any is_self=False
#thewordtrue = [True]
testingfilter = [i for i in res.json()['data'] if i['is_self'] in [True]]
len(testingfilter)
#checks out, there are no 'False' in original 'posts'...TBD if actually filtering

100

What features do I want to retain from each post?
* **selftext**    #actual text
* **subreddit**    #target
* created_utc
* is_self      #tells if text post, filter by is_self==True during scraping
* score
* title
* author
* num_comments #bc Gwen threw it in during class review?

In [64]:
pr_100 = pd.DataFrame(posts) #keys from dict become column headers
#only a few columns we care about
#can skip 'posts' and go straight to pd.DF(json_data['data'])
pr_100.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'subreddit_type', 'thumbnail', 'title', 'total_awards_received',
       'treatment_tags', 'upvote_ratio',

In [66]:
retain = ['selftext', 'subreddit', 'created_utc', 'is_self', 'score', 'title', 'author', 'num_comments']
pr_100 = pr_100[retain]
pr_100.shape

(100, 8)

In [68]:
pr_100.sort_values('created_utc').head()


,selftext,subreddit,created_utc,is_self,score,title,author,num_comments
99,\n\nI remember during my first year of colleg...,pettyrevenge,1617115680,True,1,How the tables have turned.,BestGuessGuest,79
98,A bit long TDRL at the end. First post on mobi...,pettyrevenge,1617119446,True,1,I know how to use the radio,Negative_Shake1478,23
97,Senior year of high school. Last school dance ...,pettyrevenge,1617138131,True,1,Made a girl cry after she stuck gum in my hair,The_Bee_Sneeze,44
96,"When I was working as a barista, there were tw...",pettyrevenge,1617168650,True,1,Lady waits to use the bathroom every night at ...,snailtearstains,557
95,First let me say: I fucking hate it when peopl...,pettyrevenge,1617204366,True,1,Rude lady talking on phone,Xxtratourettestriall,20
94,I work in a small office and next door in the ...,pettyrevenge,1617210329,True,1,Old lady gym demands front 1/2 of the parking ...,McNuggeteer,68
93,"I (31F) live in an apartment that, while there...",pettyrevenge,1617218775,True,1,Neighbor complains that my car smells to try t...,SchnarchendeSchwein,182
92,This story comes to me secondhand from both th...,pettyrevenge,1617226802,True,1,The customer is always right? Not at the gun s...,red_ball_express,138
91,"i’m a first year college student, and the buil...",pettyrevenge,1617227421,True,1,"College Kids Wake Me Up, So I Don’t Let Them S...",rguiry913,52
90,Just before I came out I was working as an ele...,pettyrevenge,1617227823,True,1,Gays are gross?... ok I got you!,Ragnars85,6


In [69]:
# 100 posts (not-removed) spans 24 days
utc_first = 1617115680
utc_last = 1619204046
dt.date.fromtimestamp(utc_first), dt.date.fromtimestamp(utc_last)

(datetime.date(2021, 3, 30), datetime.date(2021, 4, 23))

In [71]:
pr_100.drop_duplicates(inplace=True)

In [72]:
len(pr_100) #no duplicates

100

In [106]:
pr_100['timestamp'] = pr_100['created_utc'].map(dt.date.fromtimestamp)

In [73]:
pr_100['selftext'].str.count("removed").sum()  # successful filter during request

0.0

Use before/after to pickup where you left off scraping.  
'created_utc' integer: google any kine utc converter, plug in integer, find out date.  

## Filtering in scraping loop:
* filter by 'is_self':True
    * Did not do - found vast majority to be True in prelim requests so just filtered afterwards
* filter out 'selftext':'[removed]'  
    * yes!  in params
* remove duplicates 
    * in function, but outside of loop.  Loop will end if posts exceed determined max, but that number INCLUDES duplicates since they're not dropped til after.  Set max to 7,000 so plenty buffer to drop dupes and still have >5,000

---

## Other
* pettyrevenge: 100 posts from 3/30 - 4/23 = 24 days
* relationship_advice: can get 100 posts per day.  Set day_window same as pettyrevenge.

# For loop.  Take III.

In [112]:
def pushshift_go(subreddit, day_window, n):
    base = 'https://api.pushshift.io/reddit/search/submission'
    
    retain = ['selftext', 'subreddit', 'created_utc', 'is_self', 'score', 'title', 'author', 'num_comments']

    posts = []
    
    for i in range(1, n+1):

        params = {
        'subreddit' : subreddit,
        'size' : 100,
        'selftext:not' : '[removed]',
        'after': '{}d'.format(day_window*i) #after = searching forward, dupes will be deleted
        }
        res = requests.get(base, params)
        
        try:
            assert res.status_code == 200
            df = pd.DataFrame(res.json()['data'])[retain]
            posts.append(df)
            time.sleep(3)
        except: 
            print(f'Error: Status Code: {res.status_code}') #try-except ala Amanda

        #from class demo:
        total_scraped = sum(len(x) for x in posts)
        print(total_scraped)
        if total_scraped>7000:
            break

    #print(len(posts))

    alldata = pd.concat(posts, sort=False)
    alldata.drop_duplicates(inplace=True)
    alldata['timestamp'] = alldata['created_utc'].map(dt.date.fromtimestamp)  # from Gwen class
    
    print("Query Complete")
    
    return alldata.reset_index(drop=True)

In [53]:
# %%time
# pr3 = pushshift_go('pettyrevenge', 25, 70)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
Error: Status Code: 522
3400
3500
3600
3700
3800
3891
3991
4091
Error: Status Code: 525
4091
4191
4291
4391
4491
4591
4691
4791
4891
4991
5091
5191
5291
5391
5491
5591
5691
5791
5891
5991
6091
6191
6291
6391
6491
Error: Status Code: 522
6491
6591
6691
Query Complete
CPU times: user 4.56 s, sys: 274 ms, total: 4.84 s
Wall time: 42min 14s


In [54]:
len(pr3)

6592

In [74]:
pr3['year'] = pd.DatetimeIndex(pr3['timestamp']).year
pr3.sort_values('created_utc').head() #len 6592, earliest 2016-07-08, latest 2021-04-23

,selftext,subreddit,created_utc,is_self,score,title,author,num_comments,timestamp,year
6492,This one's short and sweet.\n\nA long time ago...,pettyrevenge,1468039320,True,1333,"Waste my time, I waste yours.",trampabroad,26,2016-07-08,2016
6493,https://imgur.com/gallery/PGl2D,pettyrevenge,1468041094,True,30,"Found this on Instagram, thought you guys migh...",Spontaneous_Mullet,13,2016-07-08,2016
6494,[deleted],pettyrevenge,1468072482,True,0,"Really? Steal *my* office? Okay, dothead....",[deleted],6,2016-07-09,2016
6495,So I am sleeping on this Saturday morning (pro...,pettyrevenge,1468080947,True,339,Happily sleeping and the phone rings,harssk,49,2016-07-09,2016
6496,[deleted],pettyrevenge,1468087635,True,131,Bad driver gets what's coming,[deleted],19,2016-07-09,2016


In [77]:
pr3.year.value_counts(normalize=True)  
#drop entries from 2016 and 2021, so easier to get matching spread in relat_advice
# did a lot of math to figure out the day_window to make this happen
# realized it's the same day_window as pettyrevenge duh.

2020    0.222694
2019    0.222239
2017    0.220874
2018    0.185983
2016    0.086924
2021    0.061286
Name: year, dtype: float64

In [120]:
pr3.is_self.value_counts()

True     6590
False       2
Name: is_self, dtype: int64

#### Remove 'is_self' == False (2), and any selftexts that were 'deleted' (~300)

In [121]:
pr3 = pr3[pr3['is_self']==True]

In [122]:
pr3.is_self.value_counts()

True    6590
Name: is_self, dtype: int64

In [135]:
pr3 = pr3[pr3['selftext']!='[deleted]']

In [136]:
pr3.to_csv('./data/pettyrev.csv', index=False)

In [137]:
pr3.to_json('./data/json_pr.json', orient='records')

# For relationship_advice, sample the same timeframe as pettyrevenge
Relationship advice is a much more active thread, but pettyrevenge went all the way back to July 2016 to get 6500 posts.  

I want roughly 90% of data from 2017-2020

In [113]:
# %%time
# ra1 = pushshift_go('relationship_advice', 25, 70)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
Query Complete
CPU times: user 4.64 s, sys: 277 ms, total: 4.91 s
Wall time: 7min 33s


In [118]:
ra1['year'] = pd.DatetimeIndex((ra1['timestamp'])).year
print(len(ra1))
ra1.head()

7000


,selftext,subreddit,created_utc,is_self,score,title,author,num_comments,timestamp,year
0,We’re not like publicly dating but we get down...,relationship_advice,1617083162,True,1,Desperately need help with my stepsister,BananaForScale69420,7,2021-03-29,2021
1,How should I tell my partner who plays games o...,relationship_advice,1617083390,True,1,How should I?,user_9012021,3,2021-03-29,2021
2,I found out on Saturday that my grandfather ha...,relationship_advice,1617083473,True,1,I don’t want to go into work.,pastabake101,10,2021-03-29,2021
3,So my girlfriends friend told me that i remind...,relationship_advice,1617083711,True,1,Advice please,RevolutionaryAD21,5,2021-03-29,2021
4,ThrowRA\nMy boyfriend(17M) and I(17F) had a ta...,relationship_advice,1617083749,True,1,How do I remember changes I want to make?,Bubbly-Appointment74,7,2021-03-29,2021


In [117]:
ra1.value_counts('year')

year
2020    1500
2018    1500
2019    1400
2017    1400
2016     800
2021     400
dtype: int64

In [119]:
ra1.is_self.value_counts()

True     6980
False      20
Name: is_self, dtype: int64

#### Remove is_self==False (20), and any 'deleted' posts (833)

In [123]:
ra1 = ra1[ra1['is_self']==True]

In [124]:
ra1.is_self.value_counts()

True    6980
Name: is_self, dtype: int64

In [140]:
ra1 = ra1[ra1.selftext!='[deleted]']

In [145]:
ra1.shape
# ra1.loc[ra1.selftext=='[deleted]']  #confirmed

(6147, 10)

In [146]:
ra1.to_csv('./data/relatadv.csv', index=False)

In [147]:
ra1.to_json('./data/json_ra.json', orient='records')